# LSTM Pos-Tagger using PyTorch

In [3]:
# importing Comet first
# Used to log data about our experiment

from comet_ml import Experiment

In [4]:
import data
import torch, os
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
from torch.nn import functional
from torch.nn import Module
from torch.utils.data import Dataset, DataLoader

device = torch.device('cpu')

##
# Loading the corpus into the following variables
#    train_dataloader           DataLoader for iterating over the training data
#    dev_dataloader             DataLoader for iterating over the development data
#    test_dataloader            DataLoader for iterating over the test data
#    vocabulary                 Vocabulary of words in the sentences in the data
#    tagset                     Vocabulary of POS tags in the data
#    pretrained_embeddings      Pretrained fasttext word embeddings 
##

train_dataloader,dev_dataloader,test_dataloader,vocabulary,tagset,pretrained_embeddings = data.load(
'corpus/de_gsd-ud-train.conllu',
'corpus/de_gsd-ud-dev.conllu',
'corpus/de_gsd-ud-test.conllu'
)

In [5]:
##
# Print information of Embeddings, Tagset and Vocab
##

print("\nPretrained Embeddings shape:")
print(pretrained_embeddings.shape)

print("\nTagset size:{}\n".format(len(tagset)))
print('Tags:\n{}'.format(tagset.lookup_tokens(range(0,len(tagset)))))

print("\nVocab size: {}".format(len(vocabulary)))

print("\nVocab sample:")
print(vocabulary.lookup_tokens(range(0,50)))




Pretrained Embeddings shape:
torch.Size([50506, 300])

Tagset size:18

Tags:
['NOUN', 'PUNCT', 'DET', 'ADP', 'PROPN', 'VERB', 'ADJ', 'ADV', 'PRON', 'AUX', 'CCONJ', 'NUM', '_', 'PART', 'SCONJ', 'X', 'SYM', 'INTJ']

Vocab size: 50506

Vocab sample:
['<unk>', '<pad>', '</s>', ',', 'der', 'die', 'in', 'und', 'dem', '-', 'von', 'zu', 'den', 'das', 'im', 'mit', 'ist', 'er', 'des', 'an', '"', ')', '(', 'ein', 'eine', 'auf', 'als', 'für', 'sich', 'wurde', 'auch', 'war', 'nach', 'bei', 'sie', 'es', 'nicht', 'aus', 'bis', 'sind', 'einer', 'werden', 'zum', 'durch', 'wird', 'ich', 'am', 'einen', 'einem', 'zur']


In [6]:
for minibatch in train_dataloader:

    print('Shape of the batch Tensor objects: {}'.format(minibatch.size()))
    print('\nFirst POS Tags:\n')
    print(' '.join(tagset.lookup_tokens(minibatch[0][1].flatten().tolist())))
    print('\nFirst Sentence:\n')
    print(' '.join(vocabulary.lookup_tokens(minibatch[0][0].flatten().tolist())))
    break

Shape of the batch Tensor objects: torch.Size([1, 2, 16])

First POS Tags:

ADV ADJ NOUN PUNCT ADJ NOUN DET NOUN PUNCT ADV VERB PRON PRON NOUN ADP PUNCT

First Sentence:

sehr gute beratung , schnelle behebung der probleme , so stelle ich mir kundenservice vor </s>


In [7]:
class LSTMPosTagger(nn.Module):
    def __init__(self,
        embeddings,
        hidden_dim,
        tagset_size):
        super(LSTMPosTagger, self).__init__()
        
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding.from_pretrained(embeddings)

        # The LSTM receives the word embedding vector a input and outputs
        # a vector of size int(hidden_dim). 
        self.lstm = nn.LSTM(self.word_embeddings.embedding_dim, self.hidden_dim)

        # A Linear layer that receives the output of the LSTM model with
        # size int(hidden_dim) and outputs a vector of size int(tagset_size) 
        self.hid_to_tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        '''
        Function to update the model parameters. Necessary for the model
        : param sentence : List of indices corresponding to the vocab in the pre trained embeddings model.
        : type List:'''
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_scores = self.hid_to_tag(lstm_out.view(len(sentence), -1))
        
        return tag_scores

In [8]:
def train_model_comet(model: Module, 
                train_data: DataLoader,
                num_epochs: int,
                optimizer_type,
                loss_function,
                learning_rate: float,
                experiment: Experiment) -> None:
    """
    Function to train a given PyTorch model with given Hyperparameters
    :param model: a pytorch model
    :param train_data: a dataloader for getting the training instances
    :param num_epochs: the number of epochs to train
    :param optimizer_type: the type of optimizer to use for training
    :param loss_function: the type of loss function to use
    :param learning_rate: the learning rate for the optimizer
    :return:
    """
    
    
    print(f'--------- Start Training ------------')

    # Important: bring model into training mode
    model.train()
    
    optimizer = optimizer_type(params=model.parameters(), lr=learning_rate)

    # Setting experiment to Train Mode.
    with experiment.train():
        step=0

        for epoch in tqdm(range(num_epochs), desc='Classifier Training\n'):
            # Cummulative loss per Epoch
            cum_loss = 0
            # Number of correct predictions
            correct = 0
            # Number of total tokens predicted
            total = 0
            
            print(f'---------- Started Epoch {epoch} -----------')

            for batch in tqdm(train_data):
                # get the inputs 
                input_attributes = batch[0][0].to(device)
                # get the corresponding labels
                gold_labels = batch[0][1].to(device)
                
                # compute model predictions with current model parameters
                model_output = model(input_attributes)
    
                # Compute Loss for current batch
                loss = loss_function(model_output, gold_labels)
                cum_loss += loss.item()
                  
                # Zero Gradient
                optimizer.zero_grad()
                loss.backward()
    
                # Update parameters
                optimizer.step()
            
                ##################################################
                # COMET
                # Compute train accuracy
                # Torch.max returns a namedtuple where of (value,indices)
                # where ```values``` is the maximum value of each rou of the
                # input tensor in the given dimension 
                _, predicted = torch.max(model_output.data, 1)

                batch_total = gold_labels.size(0)
                total += batch_total

                batch_correct = (predicted == gold_labels.data).sum()
                correct += batch_correct

                 # Log batch_accuracy to Comet.ml; step is a correction for epoch number
            step+=1
            experiment.log_metric("mean_epoch_accuracy", correct / total, step=step)
                ################################################
                
            mean_loss_per_epoch = cum_loss/len(train_data)
            experiment.log_metric('Mean_loss_per_epoch',mean_loss_per_epoch,step)
            print(mean_loss_per_epoch)

# Training the Models on Development Data 

Code used to define Hyperparameters, train the model and save the model.

**Run this only to train models**

In [ ]:
# Hyper Parameters
HIDDEN_SIZE = 300

NUM_EPOCHS = 10

LEARNING_RATE = 0.005


# Inputs
EMBEDDINGS = pretrained_embeddings

OPTIMIZER = optim.Adam

LOSS_FUNCTION = nn.functional.cross_entropy

EXPERIMENT = Experiment(project_name="LSTM-PosTagger - DevData")

# Logging the Hyper Parameters into Commet
HyperParameters = {'HiddenSize': HIDDEN_SIZE,
                   'NumEpochs': NUM_EPOCHS,
                   'LearningRate': LEARNING_RATE
                  }

EXPERIMENT.log_parameters(HyperParameters)


# Initializing the model
POS_TAGGER = LSTMPosTagger(pretrained_embeddings,HIDDEN_SIZE,len(tagset))

In [ ]:
# Using the Training Function

train_model_comet(POS_TAGGER,
            dev_dataloader,
            NUM_EPOCHS,
            OPTIMIZER,
            LOSS_FUNCTION,
            LEARNING_RATE,
            EXPERIMENT)

In [ ]:
# Saving the model. The path was changed multiple times for different models.
# wich the current settings it should save normally
torch.save(POS_TAGGER.state_dict(),'./models/LSTM_PosTagger_DEV - DevDataX.pt')

# Training the model on Development Data

Three different models were trained with different Learning Rates. 

| Hypterparameters | DevData 1 | DevData 2  | DevData 3 |
|:----------------:|---------|----------|---------|
| Learning Rate    | 0,01    | 0,001    | 0,015   |
| Hidden Size      | 300     | 300      | 300     |
| Number of Epochs | 20      | 20       | 20      |

## Evaluating Dev Models Accuracy on Test Data

Five different models were evaluated. Different Learning Rates and Hidden Sizes were used and the Mean Accuracy per Epoch and Mean Loss per Epoch were logged on Comet. The Hyperparameters are shown right before the 

Data for the models trained on the Dev Data can be found on the following link

[Comet - Dev Data](https://www.comet.ml/lflage/lstm-postagger-devdata/view/new/panels)

In [9]:
def eval_model(Data, Model):
    '''function created to return the accuracy of the models predictions on any given data
    : param Data: Iterable containing the sentences to be predicted
    : type: Dataloader
    : Model: a PyTorch Model'''
    
    Model.eval()

    with torch.no_grad():  
        correct=0
        total=0
           
        test_predictions = None
        test_targets = None
        
        for batch in Data:
            input_attributes = batch[0][0].to(device)
            # get the corresponding labels
            gold_labels = batch[0][1].to(device)
            model_output = Model(input_attributes)
            _, predicted = torch.max(model_output.data, 1)
            
            
            batch_total = gold_labels.size(0)
            total += batch_total
    
            batch_correct = (predicted == gold_labels.data).sum()
            correct += batch_correct
            
    model_accurracy = correct/total        
    print('model accuracy: {:.4f}'.format(model_accurracy))



### Model 1

| Hyperparameters | DevData 1 | 
|:----------------:|---------|
| Learning Rate    | 0,01    |
| Hidden Size      | 300     |
| Number of Epochs | 20      |

<img src='./images/Mean Accuracy per Epoch - DevData 1.svg' width="1000" height="800">

<img src='./images/Mean Loss per Epoch - DevData1.svg' width="1000" height="800">

In [13]:
# Loading the model
path = './models/LSTM_PosTagger_DEV - DevData1.pt'
lstm = LSTMPosTagger(pretrained_embeddings,300,len(tagset))
lstm.load_state_dict(torch.load(path))
lstm.eval()

if os.path.exists(path):
    eval_model(test_dataloader,lstm)
    # Obtained accuracy of: 0.8701

model accuracy: 0.8701


### Model 2

| Hyperparameters  |DevData 2 | 
|:----------------:|----------|
| Learning Rate    | 0,001    | 
| Hidden Size      | 300      | 
| Number of Epochs | 20       | 


<img src='./images/Mean Accuracy per Epoch - DevData2.svg' width="1000" height="800">

<img src='./images/Mean Loss per Epoch - DevData2.svg' width="1000" height="800">

In [14]:
# Loading the model
path = './models/LSTM_PosTagger_DEV - DevData2.pt'
lstm = LSTMPosTagger(pretrained_embeddings,300,len(tagset))
lstm.load_state_dict(torch.load(path))
lstm.eval()

if os.path.exists(path):
    eval_model(test_dataloader,lstm)
    # Obtained accuracy of: 0.8940

model accuracy: 0.8940


### Model 3

| Hyperparameters || DevData 3 |
|:----------------:||---------|
| Learning Rate    || 0,015   |
| Hidden Size      || 300     |
| Number of Epochs || 20      |

<img src='./images/Mean Accuracy per Epoch - DevData3.svg' width="1000" height="800">

<img src='./images/Mean Loss per Epoch - DevData3.svg' width="1000" height="800">

In [16]:
# Loading the model
path = './models/LSTM_PosTagger_DEV - DevData3.pt'
lstm = LSTMPosTagger(pretrained_embeddings,300,len(tagset))
lstm.load_state_dict(torch.load(path))
lstm.eval()

if os.path.exists(path):
    eval_model(test_dataloader,lstm)
    # Obtained accuracy of: 0.8802

model accuracy: 0.8802


## Different Hidden Sizes

Two other models were trained using different Hidden Sizes. The first one using hidden size 150 and the second using Hidden Size 600.

### Model 4 

  
| Hyperparameters  || DevData 4 |
|:----------------:||-----------|
| Learning Rate    ||   0,01    |
| Hidden Size      ||   150     |
| Number of Epochs ||   20      |

<img src='./images/Mean Accuracy per Epoch - DevData4.svg' width="1000" height="800">

<img src='./images/Mean Loss per Epoch - DevData4.svg' width="1000" height="800">

In [17]:
# Loading the model

path = './models/LSTM_PosTagger_DEV - DevData4.pt'
lstm = LSTMPosTagger(pretrained_embeddings,150,len(tagset))
lstm.load_state_dict(torch.load(path))
lstm.eval()

if os.path.exists(path):
    eval_model(test_dataloader,lstm)
    # Obtained accuracy of: 0.8898

model accuracy: 0.8898


### Model 5 

  
| Hyperparameters  || DevData 5 |
|:----------------:||-----------|
| Learning Rate    ||   0,01    |
| Hidden Size      ||   600     |
| Number of Epochs ||   20      |

<img src='./images/Mean Accuracy per Epoch - DevData5.svg' width="1000" height="800">

<img src='./images/Mean Loss per Epoch - DevData5.svg' width="1000" height="800">

In [18]:
# Loading the model
path = './models/LSTM_PosTagger_DEV - DevData5.pt'
lstm = LSTMPosTagger(pretrained_embeddings,600,len(tagset))
lstm.load_state_dict(torch.load(path))
lstm.eval()

if os.path.exists(path):
    eval_model(test_dataloader,lstm)
    # Obtained accuracy of: 0.8772

model accuracy: 0.8772


### Conclusion on HyperParameters

Model 1 gave us a baseline for accuracy performance (0.8701) on the test data and from this performance we can decide which Hyperparameters to use to train our model.  

Models 2 and 3 allowed us to compare the influence of the **Learning Rate** on the models performance. We found an accuracy of 0.8940 for model 2, and 0.8802 for model 3. Although both achieved a higher accuracy than the baseline model, Model2 performance was slightly better on both the test and training data (can be seen on Comet). For this reason a **Learning Rate** of 0.001 was chosen to train the final model.

When comparing models 4 and 5 accuracy on test data, we see the model has a better accuracy (0.8898) with **Hidden Size** 150, achieved on Model 4. Model 5 had a slightly better accuracy (0.8772) than the baseline model, although not as high as Model 4.

The final model was trained using 

# Computing accuracy on a given sentence

The accuracy can only be computed on sentences for which we know the correct POS-Tags. Therefore we check the accuracy of the Development model on the first sentence provided by the development Dataloarder

In [19]:
def compute_accuracy(y_true, y_pred):
    '''
    Function Defined to compute accuracy
    :param y_true: List of the correct values to be checked
    :type y_true: List
    :param y_pred: List of the predicted values
    :type y_pred: List of predicted values
    :return accuracy: Float number between 1 and 0 that represents the accuracy
    '''
    correct_predictions = 0
    # iterate over each label and check
    for true, predicted in zip(y_true, y_pred):
        if true == predicted:
            correct_predictions += 1
    # compute the accuracy
    accuracy = correct_predictions/len(y_true)
    return accuracy


In [22]:
# Loading a model and setting it to evaluation mode

model = LSTMPosTagger(pretrained_embeddings,300,len(tagset))
model.load_state_dict(torch.load('./models/LSTM_PosTagger_DEV - DevData1.pt'))
model.eval()

LSTMPosTagger(
  (word_embeddings): Embedding(50506, 300)
  (lstm): LSTM(300, 300)
  (hid_to_tag): Linear(in_features=300, out_features=18, bias=True)
)

In [23]:
# Access the first sentence of the Dataloader and checks the models accuracy

for batch in dev_dataloader:
    input_attributes = batch[0][0].to(device)
    gold_labels = batch[0][1].to(device)
    
    model_output = model(input_attributes)
    
    _, predicted = torch.max(model_output.data, 1)
    
    print('Tagging Accuracy: {}'.format(compute_accuracy(gold_labels, predicted)))
    
    break

Tagging Accuracy: 1.0


# Train model on training data
 
The model will be trained using the hyperparameters that resulted on the best accuracy for the model trained on the development data. This means we used the Hyperparameters for Model 2

In [46]:
HIDDEN_SIZE = 150

NUM_EPOCHS = 20

LEARNING_RATE = 0.001

# Inputs
EMBEDDINGS = pretrained_embeddings

OPTIMIZER = optim.Adam

LOSS_FUNCTION = nn.functional.cross_entropy

EXPERIMENT = Experiment(project_name="LSTM-PosTagger - Training Data")

COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/lflage/lstm-postagger-devdata/94f9becec1864152901864681b5db7e3
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     train_Mean_loss_per_epoch [20] : (0.08815085770042011, 0.6820052868193563)
COMET INFO:     train_mean_epoch_accuracy [20] : 0.9595257
COMET INFO:   Parameters:
COMET INFO:     HiddenSize   : 600
COMET INFO:     LearningRate : 0.01
COMET INFO:     NumEpochs    : 20
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (123 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     os packages              : 1
COMET INFO: ---------------------------
COMET INFO: Experiment is liv

In [47]:
# Logging the Hyper Parameters into Commet
HyperParameters = {'HiddenSize': HIDDEN_SIZE,
                   'NumEpochs': NUM_EPOCHS,
                   'LearningRate': LEARNING_RATE
                  }

EXPERIMENT.log_parameters(HyperParameters)

# Initializing the model
POS_TAGGER = LSTMPosTagger(pretrained_embeddings,HIDDEN_SIZE,len(tagset))

In [ ]:
# Using the train function
train_model_comet(POS_TAGGER,
            train_dataloader,
            NUM_EPOCHS,
            OPTIMIZER,
            LOSS_FUNCTION,
            LEARNING_RATE,
            EXPERIMENT)

torch.save(POS_TAGGER.state_dict(),'./LSTM_PosTagger_Train - TrainData3.pt')



Classifier Training
:   0%|          | 0/20 [00:00<?, ?it/s]


  0%|          | 0/13814 [00:00<?, ?it/s]


  0%|          | 1/13814 [00:00<37:48,  6.09it/s]

--------- Start Training ------------
---------- Started Epoch 0 -----------





  0%|          | 2/13814 [00:00<34:42,  6.63it/s]


  0%|          | 4/13814 [00:00<35:12,  6.54it/s]


  0%|          | 5/13814 [00:00<32:22,  7.11it/s]


  0%|          | 6/13814 [00:00<32:20,  7.12it/s]


  0%|          | 7/13814 [00:01<34:09,  6.74it/s]


  0%|          | 8/13814 [00:01<37:41,  6.11it/s]


  0%|          | 9/13814 [00:01<33:27,  6.88it/s]


  0%|          | 11/13814 [00:01<33:17,  6.91it/s]


  0%|          | 12/13814 [00:01<37:30,  6.13it/s]


  0%|          | 14/13814 [00:02<35:33,  6.47it/s]


  0%|          | 15/13814 [00:02<45:41,  5.03it/s]


  0%|          | 16/13814 [00:02<40:47,  5.64it/s]


  0%|          | 18/13814 [00:02<40:17,  5.71it/s]


  0%|          | 20/13814 [00:03<33:03,  6.95it/s]


  0%|          | 21/13814 [00:03<31:05,  7.39it/s]


  0%|          | 22/13814 [00:03<31:13,  7.36it/s]


  0%|          | 24/13814 [00:03<27:26,  8.38it/s]


  0%|          | 25/13814 [00:03<29:51,  7.70it/s]


  0%|          | 26/13814 [00:03<30:19,  7.58it/s]

  4%|▍         | 605/13814 [00:48<21:47, 10.10it/s]


  4%|▍         | 607/13814 [00:49<19:12, 11.46it/s]


  4%|▍         | 609/13814 [00:49<21:57, 10.02it/s]


  4%|▍         | 611/13814 [00:49<20:29, 10.74it/s]


  4%|▍         | 613/13814 [00:49<21:00, 10.47it/s]


  4%|▍         | 615/13814 [00:49<22:36,  9.73it/s]


  4%|▍         | 617/13814 [00:50<21:53, 10.04it/s]


  4%|▍         | 619/13814 [00:50<22:41,  9.69it/s]


  4%|▍         | 621/13814 [00:50<21:14, 10.35it/s]


  5%|▍         | 623/13814 [00:50<21:32, 10.21it/s]


  5%|▍         | 626/13814 [00:50<19:35, 11.22it/s]


  5%|▍         | 628/13814 [00:51<17:03, 12.88it/s]


  5%|▍         | 630/13814 [00:51<16:02, 13.69it/s]


  5%|▍         | 633/13814 [00:51<14:19, 15.33it/s]


  5%|▍         | 636/13814 [00:51<12:56, 16.98it/s]


  5%|▍         | 638/13814 [00:51<17:10, 12.79it/s]


  5%|▍         | 640/13814 [00:51<15:39, 14.02it/s]


  5%|▍         | 642/13814 [00:51<16:20, 13.43it/s]


  5%|▍         | 644/13814 [

  8%|▊         | 1167/13814 [01:38<24:17,  8.68it/s]


  8%|▊         | 1169/13814 [01:39<23:15,  9.06it/s]


  8%|▊         | 1170/13814 [01:39<27:26,  7.68it/s]


  8%|▊         | 1171/13814 [01:39<31:16,  6.74it/s]


  8%|▊         | 1172/13814 [01:39<30:50,  6.83it/s]


  8%|▊         | 1173/13814 [01:39<30:45,  6.85it/s]


  8%|▊         | 1174/13814 [01:39<30:40,  6.87it/s]


  9%|▊         | 1175/13814 [01:40<30:15,  6.96it/s]


  9%|▊         | 1178/13814 [01:40<23:35,  8.93it/s]


  9%|▊         | 1180/13814 [01:40<20:39, 10.19it/s]


  9%|▊         | 1182/13814 [01:40<21:13,  9.92it/s]


  9%|▊         | 1184/13814 [01:40<18:39, 11.28it/s]


  9%|▊         | 1186/13814 [01:40<22:24,  9.39it/s]


  9%|▊         | 1189/13814 [01:41<19:58, 10.53it/s]


  9%|▊         | 1193/13814 [01:41<16:01, 13.12it/s]


  9%|▊         | 1195/13814 [01:41<16:09, 13.02it/s]


  9%|▊         | 1197/13814 [01:41<21:22,  9.84it/s]


  9%|▊         | 1199/13814 [01:41<21:36,  9.73it/s]


  9%|▊    

...

 22%|██▏       | 3097/13814 [05:11<22:07,  8.07it/s]


 22%|██▏       | 3099/13814 [05:11<21:29,  8.31it/s]


 22%|██▏       | 3101/13814 [05:11<22:53,  7.80it/s]


 22%|██▏       | 3103/13814 [05:12<21:29,  8.30it/s]


 22%|██▏       | 3105/13814 [05:12<18:50,  9.47it/s]


 22%|██▏       | 3107/13814 [05:12<16:18, 10.94it/s]


 23%|██▎       | 3110/13814 [05:12<14:46, 12.07it/s]


 23%|██▎       | 3112/13814 [05:12<13:48, 12.92it/s]


 23%|██▎       | 3114/13814 [05:12<19:16,  9.25it/s]


 23%|██▎       | 3116/13814 [05:13<26:51,  6.64it/s]


 23%|██▎       | 3118/13814 [05:13<23:20,  7.64it/s]


 23%|██▎       | 3120/13814 [05:13<21:09,  8.43it/s]


 23%|██▎       | 3122/13814 [05:14<21:11,  8.41it/s]


 23%|██▎       | 3123/13814 [05:14<26:32,  6.71it/s]


 23%|██▎       | 3127/13814 [05:14<20:48,  8.56it/s]


 23%|██▎       | 3129/13814 [05:14<21:22,  8.33it/s]


 23%|██▎       | 3132/13814 [05:14<18:16,  9.74it/s]


 23%|██▎       | 3136/13814 [05:15<15:04, 11.80it/s]


 23%|██▎  

 25%|██▌       | 3495/13814 [06:04<22:29,  7.65it/s]


 25%|██▌       | 3496/13814 [06:04<31:18,  5.49it/s]


 25%|██▌       | 3498/13814 [06:04<26:54,  6.39it/s]


 25%|██▌       | 3499/13814 [06:05<27:43,  6.20it/s]


 25%|██▌       | 3500/13814 [06:05<31:09,  5.52it/s]


 25%|██▌       | 3501/13814 [06:05<33:42,  5.10it/s]


 25%|██▌       | 3502/13814 [06:05<31:15,  5.50it/s]


 25%|██▌       | 3503/13814 [06:05<31:52,  5.39it/s]


 25%|██▌       | 3504/13814 [06:06<34:26,  4.99it/s]


 25%|██▌       | 3505/13814 [06:06<36:55,  4.65it/s]


 25%|██▌       | 3506/13814 [06:06<34:12,  5.02it/s]


 25%|██▌       | 3508/13814 [06:06<27:17,  6.29it/s]


 25%|██▌       | 3511/13814 [06:06<22:47,  7.54it/s]


 25%|██▌       | 3513/13814 [06:07<25:47,  6.66it/s]


 25%|██▌       | 3516/13814 [06:07<20:09,  8.51it/s]


 25%|██▌       | 3518/13814 [06:07<25:25,  6.75it/s]


 25%|██▌       | 3520/13814 [06:08<27:19,  6.28it/s]


 25%|██▌       | 3521/13814 [06:08<29:58,  5.72it/s]


 25%|██▌  

 28%|██▊       | 3914/13814 [06:58<16:42,  9.88it/s]


 28%|██▊       | 3916/13814 [06:58<18:29,  8.92it/s]


 28%|██▊       | 3918/13814 [06:59<18:03,  9.14it/s]


 28%|██▊       | 3920/13814 [06:59<15:19, 10.76it/s]


 28%|██▊       | 3922/13814 [06:59<17:22,  9.49it/s]


 28%|██▊       | 3924/13814 [07:00<27:44,  5.94it/s]


 28%|██▊       | 3926/13814 [07:00<23:49,  6.91it/s]


 28%|██▊       | 3927/13814 [07:00<27:49,  5.92it/s]


 28%|██▊       | 3928/13814 [07:00<27:07,  6.08it/s]


 28%|██▊       | 3930/13814 [07:01<27:18,  6.03it/s]


 28%|██▊       | 3932/13814 [07:01<21:44,  7.58it/s]


 28%|██▊       | 3934/13814 [07:01<21:40,  7.59it/s]


 28%|██▊       | 3935/13814 [07:01<21:18,  7.73it/s]


 28%|██▊       | 3936/13814 [07:01<26:01,  6.33it/s]


 29%|██▊       | 3938/13814 [07:02<25:00,  6.58it/s]


 29%|██▊       | 3939/13814 [07:02<28:13,  5.83it/s]


 29%|██▊       | 3940/13814 [07:02<26:29,  6.21it/s]


 29%|██▊       | 3942/13814 [07:02<25:35,  6.43it/s]


 29%|██▊  

 32%|███▏      | 4354/13814 [07:53<22:21,  7.05it/s]


 32%|███▏      | 4355/13814 [07:54<30:13,  5.22it/s]


 32%|███▏      | 4356/13814 [07:54<38:15,  4.12it/s]


 32%|███▏      | 4357/13814 [07:54<33:12,  4.75it/s]


 32%|███▏      | 4358/13814 [07:54<29:13,  5.39it/s]


 32%|███▏      | 4360/13814 [07:54<25:57,  6.07it/s]


 32%|███▏      | 4362/13814 [07:55<25:16,  6.23it/s]


 32%|███▏      | 4363/13814 [07:55<27:19,  5.76it/s]


 32%|███▏      | 4365/13814 [07:55<23:05,  6.82it/s]


 32%|███▏      | 4368/13814 [07:55<21:02,  7.48it/s]


 32%|███▏      | 4369/13814 [07:56<22:25,  7.02it/s]


 32%|███▏      | 4370/13814 [07:56<22:51,  6.89it/s]


 32%|███▏      | 4373/13814 [07:56<18:21,  8.57it/s]


 32%|███▏      | 4375/13814 [07:56<19:22,  8.12it/s]


 32%|███▏      | 4377/13814 [07:56<18:23,  8.55it/s]


 32%|███▏      | 4378/13814 [07:57<18:35,  8.46it/s]


 32%|███▏      | 4380/13814 [07:57<15:24, 10.21it/s]


 32%|███▏      | 4382/13814 [07:57<14:13, 11.05it/s]


 32%|███▏ 

 35%|███▍      | 4800/13814 [08:47<13:15, 11.33it/s]


 35%|███▍      | 4803/13814 [08:47<11:27, 13.10it/s]


 35%|███▍      | 4805/13814 [08:47<11:14, 13.35it/s]


 35%|███▍      | 4807/13814 [08:47<10:48, 13.90it/s]


 35%|███▍      | 4810/13814 [08:47<10:20, 14.51it/s]


 35%|███▍      | 4812/13814 [08:47<10:42, 14.02it/s]


 35%|███▍      | 4814/13814 [08:48<17:19,  8.66it/s]


 35%|███▍      | 4816/13814 [08:48<21:12,  7.07it/s]


 35%|███▍      | 4818/13814 [08:48<17:10,  8.73it/s]


 35%|███▍      | 4820/13814 [08:49<19:01,  7.88it/s]


 35%|███▍      | 4822/13814 [08:49<18:42,  8.01it/s]


 35%|███▍      | 4824/13814 [08:49<20:07,  7.44it/s]


 35%|███▍      | 4828/13814 [08:49<15:30,  9.66it/s]


 35%|███▍      | 4831/13814 [08:49<13:10, 11.36it/s]


 35%|███▍      | 4833/13814 [08:50<15:58,  9.37it/s]


 35%|███▌      | 4835/13814 [08:50<16:41,  8.97it/s]


 35%|███▌      | 4837/13814 [08:50<14:31, 10.30it/s]


 35%|███▌      | 4839/13814 [08:50<18:25,  8.12it/s]


 35%|███▌ 

# Evaluating Model on the Test Data

The training data for this model can be accessed through the following [link](https://www.comet.ml/lflage/lstm-postagger-training-data/view/new/panels)

| Hyperparameters  || TrainData 1|
|:----------------:||-----------|
| Learning Rate    ||   0,001   |
| Hidden Size      ||   150     |
| Number of Epochs ||   20      |

<img src='./images/Mean Accuracy per Epoch - TrainData.svg' width="1000" height="800">

<img src='./images/Mean Loss per Epoch - TrainData.svg' width="1000" height="800">

In [24]:
path = './models/LSTM_PosTagger_Train - TrainData1.pt'
lstm = LSTMPosTagger(pretrained_embeddings,150,len(tagset))
lstm.load_state_dict(torch.load(path))
lstm.eval()


if os.path.exists(path):
    eval_model(test_dataloader,lstm)

model accuracy: 0.9036


# Conclusions

Evaluating the models created with **Development Data** on the **Test Data** might not be the standard procedure, but it provided a consistent baseline and comparable parameters to train the model on the **Training Data**.

The final accuracy obtained on the **Test Data** was of 90.36% which is comparable with the best scores obtained for the most efficient representative models and the baseline neural models. 